In [2]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import re
html_s = urllib.request.urlopen('https://en.wikipedia.org/wiki/SABMiller_brands').read()
soup_s = BeautifulSoup(html_s, 'html.parser')
brandsWiki_s = re.findall('<li>(.*)<', soup_s.decode())
b_1 = map(lambda x: x.split(' (')[0].replace('</li>',''), brandsWiki_s)
b_1 = list(b_1)
b_1.index('<a class="mw-redirect" href="/wiki/Viva_(beverage)" title="Viva')
b_1 = b_1[:195]
for i in range(len(b_1)):
    if b_1[i][0] == '<':
        #print(b_1[i])
        b_1[i] = 0

#print(b_1.index('Appletiser'))
b_2 = b_1[:170]
b_2.extend(('Carling Black Label','Castle Lager','Castle Lite','Castle Milk Stout','Castle Free','Miller Genuine Draft','Peroni','Fosters','Chibuku Opaque Beer','Pilsner Urquell','Dreher Sörgyárak','miller'))
b_2 = list(map(lambda x: x.split('<')[0], list(filter(lambda a: a != 0, b_2))))
b_2 = list(set(b_2))

html_i = urllib.request.urlopen('https://en.wikipedia.org/wiki/AB_InBev_brands').read()
soup_i = BeautifulSoup(html_i, 'html.parser')
b_ = re.findall('<li>(.*)<', soup_i.decode())
b_ = b_[:411]
for i in range(len(b_)):
    if b_[i][0] == '<':
        b_[i] = b_[i].split('>')[1]
#print(b_)
       
b_3 = map(lambda x: x.split(' (')[0].split('<')[0].replace('\\', ''), b_)
b_3 = list(b_3)
#print(b_3.index('Labatt '))
b_3[326] = 'Lucky Lager'
b_3 = list(set(b_3))
#print(len(b_3))
#len(b_3)

b_2 = map(lambda x:x.upper(), b_2)
brands_s = list(b_2)

b_3 = map(lambda x:x.upper(), b_3)
brands_i = list(b_3)
brands_i

['DIEBELS DIMIX',
 'AREQUIPEÑA',
 'PRESIDENTE LIGHT',
 'CASS',
 'ROHAN VESELYI MONACH',
 'SPATEN MÜNCHNER HELL',
 'CASS FRESH',
 'PIEDBOEUF BLOND',
 'HERTOG JAN',
 "FOSTER'S",
 'ROHAN TRADYTSIYNE',
 'DIEBELS LIGHT',
 'DIMIX',
 'LEFFE RUBY',
 'BORSODI POLO',
 'BELLE-VUE KRIEK EXTRA',
 'PIVARA TREBJESA',
 'BUDWEISER SELECT 55',
 'TOMISLAV',
 'STEELER',
 'WILD BLACK',
 'KK',
 "BECK'S",
 'BUDWEISER SELECT',
 'FOUR PEAKS',
 'BRUTAL FRUIT',
 'DRY BEER',
 'TOLSTIAK SVETLOYE',
 'LAKEPORT ICE',
 'PACEÑA',
 'GUARANÁ ANTARCTICA',
 'HUARI',
 'LABATT ICE',
 'ZAGREBAČKA PIVOVARA',
 'LÖWENBRÄU RADLER',
 'BOŽIĆNO PIVO',
 'LABATT GENUINE',
 'BAISHA',
 'LÖWENBRÄU LÖWEN WEISSE',
 'BÁLTICA',
 'CAMDEN TOWN BREWERY',
 'DEVILS BACKBONE BREWING COMPANY',
 'NEWLANDS SPRING',
 "ALEXANDER KEITH'S",
 'DOMMELSCH DOMINATOR',
 'FRANZISKANER HEFE-WEISSBIER LEICHT',
 'JUPILER',
 'HOP HOUND AMBER WHEAT',
 'CHERNIHIVSKE N/A',
 'NIK COOL',
 'CARLTON &AMP; UNITED BREWERIES',
 'BUSCH LIGHT',
 'LABATT .5',
 'CASCADE BREWERY

In [ ]:
import pandas as pd
df = pd.read_csv("products.tsv", delimiter = "\t", encoding = "cp1252", header = 0)

df['TargetName'] = df['brand_descr'].apply(lambda x: 'SABMiller PLC' if (x in brands_s) else 'None')
dfTarget = df[df['TargetName'] == 'SABMiller PLC']
product_groups_s = list(set(dfTarget['product_group_descr']))
groupfreq_s = {}
for group in product_groups_s:
    freq_s = len(list(set(dfTarget['brand_descr'][dfTarget['product_group_descr'] == group])))
    groupfreq_s[group] = freq_s
    
groupfreq_s = {key:val for key, val in groupfreq.items() if val != 0}

df_s = pd.DataFrame([groupfreq_s]).T.rename(columns={0: '# of products (Target)'}).rename_axis('product group')
dfSAB = df_s.sort_values('# of products (Target)', ascending=False)
# dfSAB.loc['source'] = 'https://en.wikipedia.org/wiki/SABMiller_brands'

df['AcquirorName'] = df['brand_descr'].apply(lambda x: 'Anheuser-Busch Inbev SA/NV' if (x in brands_i) else 'None')
dfAcquiror = df[df['AcquirorName'] == 'Anheuser-Busch Inbev SA/NV']
product_groups_i = list(set(dfAcquiror['product_group_descr']))
groupfreq_i = {}
for group in product_groups_i:
    freq_i = len(list(set(dfAcquiror['brand_descr'][dfAcquiror['product_group_descr'] == group])))
    groupfreq_i[group] = freq_i

#print(groupfreq_i)

groupfreq_i = {key:val for key, val in groupfreq.items() if val != 0}

df_i = pd.DataFrame([groupfreq_i]).T.rename(columns={0: '# of products (Acquiror)'}).rename_axis('product group')
dfInbev = df_i.sort_values('# of products (Acquiror)', ascending=False)
#dfInbev.loc['source'] = 'https://en.wikipedia.org/wiki/AB_InBev_brands'

dfSABInbev = pd.merge(dfInbev, dfSAB, how='outer', left_index=True, right_index=True)
dfSABInbev.to_csv("SABInbev_draft1.tsv")